In [ ]:
# default_exp models

# Models

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning.core import LightningModule
from pytorch_lightning.metrics import functional as FM

C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:23: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [ ]:
#export
from isic.dataset import SkinDataModule
from isic.callback.hyperlogger import HyperparamsLogger
from isic.callback.logtable import LogTableMetricsCallback
from isic.callback.mixup import MixupDict

In [ ]:
#export
class ResnetModel(LightningModule):
    def __init__(self):
        super().__init__()
#         self.save_hyperparameters()
        self.resnet = models.resnet50(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 7)
        self.loss_func = F.cross_entropy

    def forward(self, x):
        return self.resnet(x)

    def training_step(self, batch, batch_idx):
        x, y = batch['img'], batch['label']
        y_hat = self(x)
        loss = self.loss_func(y_hat, y)
        acc = FM.accuracy(y_hat, y, num_classes=7)
        result = pl.TrainResult(minimize=loss)
        result.log('train_loss', loss)
        result.log('train_acc', acc, prog_bar=True)
        return result

    def validation_step(self, batch, batch_idx):
        x, y = batch['img'], batch['label']
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = FM.accuracy(y_hat, y, num_classes=7)
        result = pl.EvalResult(checkpoint_on=loss)
        result.log('val_loss', loss, prog_bar=True) 
        result.log('val_acc', acc, prog_bar=True)
        return result

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-2)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=1e-2, steps_per_epoch=140, epochs=10)
        sched = {
            'scheduler': scheduler, # The LR schduler
            'interval': 'step', # The unit of the scheduler's step size
            'frequency': 1, # The frequency of the scheduler
            'reduce_on_plateau': False, # For ReduceLROnPlateau scheduler
        }
        return [opt], [sched]

In [ ]:
message_formater = "You have set {0} number of classes if different from predicted {0} and target {0} number of classes"
warnings.filterwarnings("ignore", message_formater.format("(.*)"), category=UserWarning)

In [ ]:
dm = SkinDataModule()
dm.prepare_data()
dm.setup('fit')

In [ ]:
# init model
model = ResnetModel()
hp_log = HyperparamsLogger()

# most basic trainer, uses good defaults
trainer = pl.Trainer(max_epochs=3, callbacks=[LogTableMetricsCallback(), hp_log, MixupDict()], fast_dev_run=True)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, dm)


  | Name   | Type   | Params
----------------------------------
0 | resnet | ResNet | 23 M  


C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:23: UserWarning: 
                    When using EvalResult(early_stop_on=X) or TrainResult(early_stop_on=X) the
                    'monitor' key of ModelCheckpoint has no effect.
                    Remove ModelCheckpoint(monitor='loss) to fix')
                
  warnings.warn(*args, **kwargs)


epoch,train_loss,train_acc,val_loss,val_acc
1,1.987,0.109375,2.134377,0.000000


1

In [ ]:
model = ResnetModel(lr=1,ere=123)

In [ ]:
model.hparams

"ere": 123
"lr":  1

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 4636), started 0:02:09 ago. (Use '!kill 4636' to kill it.)

In [ ]:
# #
# class ResnetModel(LightningModule):
#     def __init__(self):
#         super().__init__()
# #         self.save_hyperparameters()
#         self.resnet = models.resnet50(pretrained=True)
#         num_ftrs = self.resnet.fc.in_features
#         self.resnet.fc = nn.Linear(num_ftrs, 7)
#         self.loss_func = F.cross_entropy

#     def forward(self, x):
#         return self.resnet(x)

#     def training_step(self, batch, batch_idx):
#         print(batch)
#         if hasattr(self, 'enhanced_batch'):
#             print('hehe')
#             batch = self.enhanced_batch
#         x, y = batch
#         y_hat = self(x)
#         loss = self.loss_func(y_hat, y)
#         acc = FM.accuracy(y_hat, y, num_classes=7)
#         result = pl.TrainResult(minimize=loss)
#         result.log('train_loss', loss)
#         result.log('train_acc', acc, prog_bar=True)
#         return result

#     def validation_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = F.cross_entropy(y_hat, y)
#         acc = FM.accuracy(y_hat, y, num_classes=7)
#         result = pl.EvalResult(checkpoint_on=loss)
#         result.log('val_loss', loss, prog_bar=True) 
#         result.log('val_acc', acc, prog_bar=True)
#         return result

#     def configure_optimizers(self):
#         opt = torch.optim.Adam(self.parameters(), lr=1e-2)
#         scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=1e-2, steps_per_epoch=140, epochs=10)
#         sched = {
#             'scheduler': scheduler, # The LR schduler
#             'interval': 'step', # The unit of the scheduler's step size
#             'frequency': 1, # The frequency of the scheduler
#             'reduce_on_plateau': False, # For ReduceLROnPlateau scheduler
#         }
#         return [opt], [sched]